In [4]:
import numpy as np
from osgeo import ogr, gdal, osr, gdal_array
import rasterio
import json


import matplotlib.pyplot as plt
import pandas as pd
import os.path
import re


import rasterio
from rasterio.enums import Resampling

#### DEM upscaling

In [5]:
area_dem = '/home/shrayank_mistry/Modules/project-mum-pune/dem_clipped.tif'
dem = rasterio.open(area_dem, count = 1)

dem = np.array(dem.read(1))
print(dem.shape)

(2721, 3610)


In [10]:
# Upscaling DEM
# whole-numbers indicate upscaling, fractions indicate downscaling
upscale_factor = 3


with rasterio.open('/home/shrayank_mistry/Modules/project-mum-pune/dem_clipped.tif') as dataset:

    # resample data to target shape
    data = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height * upscale_factor),
            int(dataset.width * upscale_factor)
        ),
        resampling=Resampling.bilinear
    )

    # scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )

#### Load LULC class mask

In [11]:
area_cover = '/home/shrayank_mistry/Modules/project-mum-pune/mask.tif'
area = rasterio.open(area_cover, count = 1)

area = np.array(area.read(1))

area = np.swapaxes(area, 1, 0)
print(area.shape)

(10500, 8273)


In [12]:
up_dem = data[0]
up_dem.shape

(8163, 10830)

#### Matching DEM and Mask Resolution

In [13]:
dem_data, mask_data = up_dem, area
dem_data = dem_data.T


print(dem_data.shape, mask_data.shape)
width, height = 10500, 8163

dem_data, mask_data = dem_data[:width,:height], mask_data[:width, :height]
print(dem_data.shape, mask_data.shape)

(10830, 8163) (10500, 8273)
(10500, 8163) (10500, 8163)


##### AHP Mapping

In [14]:
ahp_map = {

'urban': 0.29,
'farms': 0.239,
'dense-forest': 0.207,
'water': 0.13,
'fallow': 0.067,
'sparse-forest': 0.049,
'barren-land': 0.019,
'unclassified':7,

}

class_map = {

    0: 'unclassified',
    1: 'water',
    2: 'dense-forest',
    3: 'sparse-forest',
    4: 'barren-land',
    5: 'urban',
    6: 'farms',
    7: 'fallow',

}

#### Set weights for LULC classes using AHP

In [15]:
def set_weights(c):
    c_str = class_map.get(c)
    return ahp_map[c_str]

In [16]:
set_weights_vtr = np.vectorize(set_weights)
mask_data = set_weights_vtr(mask_data)

In [17]:
np.unique(mask_data, return_counts = True)

(array([0.019, 0.049, 0.067, 0.13 , 0.207, 0.239, 0.29 , 7.   ]),
 array([19822048, 17097029, 28893954,  7807154,  3858436,  5940554,
         2291737,      588]))

#### Getting Source and Destination Points to Co-ordinates

In [19]:
from osgeo import ogr
import json

file = ogr.Open("/home/shrayank_mistry/Modules/project-mum-pune/route-files/source-point.shp")
source_shp = file.GetLayer(0)

feature = source_shp.GetFeature(0)
source_shp = feature.ExportToJson()

# source_shp
source_shp = json.loads(source_shp)
start_ext = source_shp['geometry']['coordinates']


file = ogr.Open("/home/shrayank_mistry/Modules/project-mum-pune/route-files/destination-point.shp")
destination_shp = file.GetLayer(0)

feature = destination_shp.GetFeature(0)
destination_shp = feature.ExportToJson()

destination_shp

# destination_shp
destination_shp = json.loads(destination_shp)
end_ext = destination_shp['geometry']['coordinates']

In [20]:
print(start_ext, end_ext)

[300378.7763074944, 2104126.3099621385] [366123.3883685307, 2063699.8154670105]


#### Getting Raster Extent

In [21]:
path = '/home/shrayank_mistry/Modules/project-mum-pune/raster.tif' 

data = rasterio.open(path)
print(data.bounds)

extent = data.bounds

left, bottom, right, top = extent[0], extent[1], extent[2], extent[3]
print(left, bottom, right, top)

BoundingBox(left=282680.0, bottom=2035820.0, right=387680.0, top=2118550.0)
282680.0 2035820.0 387680.0 2118550.0


In [22]:
width = round(right - left)
height = round(top - bottom)

print("Width and Height of Raster")
print(width, height)

Width and Height of Raster
105000 82730


##### Convert Co-ordinates to Pixels 

In [23]:
# width = round(right - left)
# height = round(top - bottom)

# print(width, height)

# start_pixel, end_pixel = [0, 0], [0, 0]
# start_flag, end_flag = True, True

# for i in range(height):
#     for j in range(width):

#         if (not start_flag) and (not end_flag):
#             break
        
#         # print(round(left + j), round(top - i))
#         if (start_flag and ((round(left + j)) == round(start_ext[0])) and ((round(top - i)) == round(start_ext[1]))):
#             start_pixel = [i, j]
#             start_flag = False

        
#         if (end_flag and ((round(left + j)) == round(end_ext[0])) and ((round(top - i)) == round(end_ext[1]))):
#             end_pixel = [i, j]
#             end_flag = False

In [24]:
start_pixel, end_pixel = [14424, 17699], [54850, 83443]

#### Downscaling Pixels

In [25]:
start_pixel = np.array(start_pixel)
start_pixel = start_pixel / 10

start_pixel = list(np.rint(start_pixel))
start_pixel = list(np.array(start_pixel, dtype = 'int'))

end_pixel = np.array(end_pixel)
end_pixel = end_pixel / 10

end_pixel = list(np.rint(end_pixel))
end_pixel = list(np.array(end_pixel, dtype = 'int'))